In [1]:
pip install pandas pymysql


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import pymysql

# Função para conectar ao banco de dados MySQL
def connect_to_mysql(host, user, password, database):
    try:
        connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )
        return connection
    except pymysql.MySQLError as e:
        print(f"Erro ao conectar ao MySQL: {e}")
        return None

# Configurações de conexão
host = 'localhost'
user = 'root'
password = '123456'
database = 'veterinaria'

# Conectando ao banco de dados
connection = connect_to_mysql(host, user, password, database)

In [4]:
# Carregar os dados das tabelas em DataFrames
venda = pd.read_sql("SELECT * FROM Venda", connection)
funcionario = pd.read_sql("SELECT * FROM funcionario", connection)
cliente = pd.read_sql("SELECT * FROM Cliente", connection)
pagamento = pd.read_sql("SELECT * FROM pagamento", connection)
itens_venda = pd.read_sql("SELECT * FROM itens_venda", connection)
produto = pd.read_sql("SELECT * FROM Produto", connection)
animal = pd.read_sql("SELECT * FROM Animal", connection)
cliente_produto = pd.read_sql("SELECT * FROM cliente_produto", connection)
cliente_servico = pd.read_sql("SELECT * FROM cliente_servico", connection)
endereco = pd.read_sql("SELECT * FROM endereco", connection)
fornecedor = pd.read_sql("SELECT * FROM fornecedor", connection)
itens_venda = pd.read_sql("SELECT * FROM itens_venda", connection)
servico = pd.read_sql("SELECT * FROM servico", connection)
# Fechando a conexão
if connection:
    connection.close()


C:\Users\WTINFO PC\AppData\Local\Temp\ipykernel_11412\4088197772.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  venda = pd.read_sql("SELECT * FROM Venda", connection)
C:\Users\WTINFO PC\AppData\Local\Temp\ipykernel_11412\4088197772.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  funcionario = pd.read_sql("SELECT * FROM funcionario", connection)
C:\Users\WTINFO PC\AppData\Local\Temp\ipykernel_11412\4088197772.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cliente = pd.read_sql("SELECT * FROM Cliente", connectio

In [5]:
# Verificação de colunas para debug
def check_columns(df, expected_columns):
    missing_columns = [col for col in expected_columns if col not in df.columns]
    if missing_columns:
        print(f"Colunas ausentes em {df.name}: {missing_columns}")
    else:
        print(f"Todas as colunas presentes em {df.name}")

# Nomear os DataFrames para facilitar o debug
venda.name = "Venda"
funcionario.name = "Funcionario"
cliente.name = "Cliente"
pagamento.name = "Pagamento"
itens_venda.name = "Itens_venda"
produto.name = "Produto"
animal.name = "Animal"

# Verificar colunas
check_columns(venda, ['Funcionario_cpf', 'Cliente_cpf', 'id', 'pagamento_id', 'Data_hora'])
check_columns(funcionario, ['cpf', 'nome'])
check_columns(cliente, ['cpf', 'nome'])
check_columns(pagamento, ['id', 'forma_pagamento'])
check_columns(itens_venda, ['Venda_idVenda', 'Produto_id'])
check_columns(produto, ['id', 'descricao', 'quantidade', 'preco'])
check_columns(animal, ['nome', 'especie', 'idade', 'peso', 'Cliente_cpf'])

Todas as colunas presentes em Venda
Todas as colunas presentes em Funcionario
Todas as colunas presentes em Cliente
Todas as colunas presentes em Pagamento
Todas as colunas presentes em Itens_venda
Todas as colunas presentes em Produto
Todas as colunas presentes em Animal


In [6]:
# Junção múltipla de 6 tabelas: Venda, Funcionario, Cliente, Pagamento, Itens_venda, Produto
result1 = pd.merge(venda, funcionario, left_on='Funcionario_cpf', right_on='cpf', how='left', suffixes=('', '_funcionario')) \
            .merge(cliente, left_on='Cliente_cpf', right_on='cpf', how='left', suffixes=('', '_cliente')) \
            .merge(pagamento, left_on='pagamento_id', right_on='id', how='left', suffixes=('', '_pagamento')) \
            .merge(itens_venda, left_on='id', right_on='Venda_idVenda', how='left') \
            .merge(produto, left_on='Produto_id', right_on='id', how='left', suffixes=('', '_produto')) \
            [['Funcionario_cpf', 'nome', 'Cliente_cpf', 'cpf_cliente', 'forma_pagamento', 'Data_hora', 'descricao', 'quantidade', 'preco']]

print("Resultado da junção múltipla:")
print(result1.head())


Resultado da junção múltipla:
   Funcionario_cpf            nome  Cliente_cpf  cpf_cliente  \
0        123456789      João Silva  12057103021  12057103021   
1        987654321    Maria Santos  12057193021  12057193021   
2        456789123  Pedro Oliveira  14719263038  14719263038   
3        789123456     Ana Pereira  18315064088  18315064088   

     forma_pagamento           Data_hora     descricao  quantidade   preco  
0  Cartão de Crédito 2023-05-19 10:30:00  Ração Canina        50.0  100.00  
1   Cartão de Débito 2023-05-19 11:00:00           NaN         NaN     NaN  
2           Dinheiro 2023-05-19 11:30:00  Ração Felina        30.0   80.00  
3                Pix 2023-05-19 12:00:00           NaN         NaN     NaN  


In [7]:
# Group by com agregações
result2 = pd.merge(venda, funcionario, left_on='Funcionario_cpf', right_on='cpf', how='left', suffixes=('', '_funcionario')) \
            .merge(cliente, left_on='Cliente_cpf', right_on='cpf', how='left', suffixes=('', '_cliente')) \
            .merge(pagamento, left_on='pagamento_id', right_on='id', how='left', suffixes=('', '_pagamento')) \
            .merge(itens_venda, left_on='id', right_on='Venda_idVenda', how='left') \
            .merge(produto, left_on='Produto_id', right_on='id', how='left', suffixes=('', '_produto')) \
            .groupby(['Funcionario_cpf', 'forma_pagamento']) \
            .agg(Nome_Funcionario=('nome', 'first'), 
                 Qdte_Total=('quantidade', 'sum'), 
                 Valor_Total=('preco', 'sum')) \
            .reset_index()

print("\nResultado do Group by:")
print(result2.head())



Resultado do Group by:
   Funcionario_cpf    forma_pagamento Nome_Funcionario  Qdte_Total Valor_Total
0        123456789  Cartão de Crédito       João Silva        50.0      100.00
1        456789123           Dinheiro   Pedro Oliveira        30.0       80.00
2        789123456                Pix      Ana Pereira         0.0           0
3        987654321   Cartão de Débito     Maria Santos         0.0           0


In [16]:
# Subconsulta EXISTS: Funcionários que realizaram vendas
funcionarios_com_vendas = venda['Funcionario_cpf'].unique()
result5 = funcionario[funcionario['cpf'].isin(funcionarios_com_vendas)][['nome']]

print("\nResultado da subconsulta EXISTS (funcionários com vendas):")
print(result5.head())



Resultado da subconsulta EXISTS (funcionários com vendas):
              nome
2       João Silva
6   Pedro Oliveira
10     Ana Pereira
12    Maria Santos


In [13]:
# LEFT JOIN: Mostrar todas as vendas, mesmo aquelas que não possuem itens
result7 = venda \
    .merge(itens_venda, left_on='id', right_on='Venda_idVenda', how='left') \
    [['id', 'Funcionario_cpf', 'Cliente_cpf', 'pagamento_id', 'Data_hora', 'Produto_id']]

print("\nResultado do LEFT JOIN:")
print(result7.head())



Resultado do LEFT JOIN:
   id  Funcionario_cpf  Cliente_cpf  pagamento_id           Data_hora  \
0   1        123456789  12057103021             1 2023-05-19 10:30:00   
1   2        987654321  12057193021             2 2023-05-19 11:00:00   
2   3        456789123  14719263038             3 2023-05-19 11:30:00   
3   4        789123456  18315064088             4 2023-05-19 12:00:00   

   Produto_id  
0         1.0  
1         NaN  
2         2.0  
3         NaN  


In [17]:
# Subconsulta NOT EXISTS: Produtos que nunca foram vendidos
produtos_vendidos = itens_venda['Produto_id'].unique()
result6 = produto[~produto['id'].isin(produtos_vendidos)][['id', 'descricao']]

print("\nResultado da subconsulta NOT EXISTS (produtos não vendidos):")
print(result6.head())



Resultado da subconsulta NOT EXISTS (produtos não vendidos):
   id              descricao
2   3          Medicamento A
3   4    Brinquedo para Cães
4   5     Coleira Antipulgas
5   6      Shampoo para Cães
6   7  Arranhador para Gatos


In [24]:
# Consulta com 2 funções de agregação: Contar o número total de vendas realizadas por cada cliente
result8 = venda \
    .merge(cliente, left_on='Cliente_cpf', right_on='cpf', how='left') \
    .groupby(['Cliente_cpf', 'nome']) \
    .agg(Total_Vendas=('id', 'count')) \
    .reset_index() \
    .rename(columns={'nome': 'Nome_Cliente'})

print("\nResultado da contagem de vendas por cliente:")
print(result8.head())



Resultado da contagem de vendas por cliente:
   Cliente_cpf Nome_Cliente  Total_Vendas
0  12057103021       Bumbwe             1
1  12057193021       Forond             1
2  14719263038       Reaion             1
3  18315064088        Geurt             1


In [25]:
# Consulta com 2 funções de agregação: Contar o número total de vendas realizadas por cada cliente
result8 = venda \
    .merge(cliente, left_on='Cliente_cpf', right_on='cpf', how='left') \
    .groupby(['Cliente_cpf', 'nome']) \
    .agg(Total_Vendas=('id', 'count')) \
    .reset_index() \
    .rename(columns={'nome': 'Nome_Cliente'})

print("\nResultado da contagem de vendas por cliente:")
print(result8.head())



Resultado da contagem de vendas por cliente:
   Cliente_cpf Nome_Cliente  Total_Vendas
0  12057103021       Bumbwe             1
1  12057193021       Forond             1
2  14719263038       Reaion             1
3  18315064088        Geurt             1


In [30]:
# Junção múltipla de 3 tabelas: Venda, Funcionario, Cliente, Pagamento, Itens_venda, Produto
result1 = pd.merge(venda, funcionario, left_on='Funcionario_cpf', right_on='cpf', how='left') \
            .merge(cliente, left_on='Cliente_cpf', right_on='cpf', how='left', suffixes=('_funcionario', '_cliente')) \
            .merge(pagamento, left_on='pagamento_id', right_on='id', how='left') \
            .merge(itens_venda, left_on='id_x', right_on='Venda_idVenda', how='left') \
            .merge(produto, left_on='Produto_id', right_on='id', how='left', suffixes=('_itens_venda', '_produto')) \
            [['Funcionario_cpf', 'nome_funcionario', 'Cliente_cpf', 'cpf_cliente', 'forma_pagamento', 'Data_hora', 'descricao', 'quantidade', 'preco']]

print("Resultado da junção múltipla:")
print(result1.head())


Resultado da junção múltipla:
   Funcionario_cpf nome_funcionario  Cliente_cpf  cpf_cliente  \
0        123456789       João Silva  12057103021  12057103021   
1        987654321     Maria Santos  12057193021  12057193021   
2        456789123   Pedro Oliveira  14719263038  14719263038   
3        789123456      Ana Pereira  18315064088  18315064088   

     forma_pagamento           Data_hora     descricao  quantidade   preco  
0  Cartão de Crédito 2023-05-19 10:30:00  Ração Canina        50.0  100.00  
1   Cartão de Débito 2023-05-19 11:00:00           NaN         NaN     NaN  
2           Dinheiro 2023-05-19 11:30:00  Ração Felina        30.0   80.00  
3                Pix 2023-05-19 12:00:00           NaN         NaN     NaN  


In [18]:
# Group by com agregações
result2 = pd.merge(venda, funcionario, left_on='Funcionario_cpf', right_on='cpf', how='left') \
            .merge(pagamento, left_on='pagamento_id', right_on='id', how='left') \
            .merge(itens_venda, left_on='id_x', right_on='Venda_idVenda', how='left') \
            .merge(produto, left_on='Produto_id', right_on='id', how='left') \
            .groupby(['Funcionario_cpf', 'forma_pagamento']) \
            .agg(Nome_Funcionario=('nome', 'first'), 
                 Qdte_Total=('quantidade', 'sum'), 
                 Valor_Total=('preco', 'sum')) \
            .reset_index()

print("\nResultado do Group by:")
print(result2.head())



Resultado do Group by:
   Funcionario_cpf    forma_pagamento Nome_Funcionario  Qdte_Total Valor_Total
0        123456789  Cartão de Crédito       João Silva        50.0      100.00
1        456789123           Dinheiro   Pedro Oliveira        30.0       80.00
2        789123456                Pix      Ana Pereira         0.0           0
3        987654321   Cartão de Débito     Maria Santos         0.0           0


In [37]:
# Group by e Order by
result3 = pd.merge(venda, cliente, left_on='Cliente_cpf', right_on='cpf', how='left') \
            .merge(itens_venda, left_on='id', right_on='Venda_idVenda', how='left') \
            .merge(produto, left_on='Produto_id', right_on='id', how='left') \
            .groupby(['Cliente_cpf', 'cpf']) \
            .agg(Qdte_Total=('quantidade', 'sum'), 
                 Valor_Total=('preco', 'sum')) \
            .reset_index()

# Convertendo Valor_Total para tipo numérico (caso necessário)
result3['Valor_Total'] = pd.to_numeric(result3['Valor_Total'])

# Ordenando pelo Valor_Total em ordem decrescente
result3 = result3.sort_values(by='Valor_Total', ascending=False)

print("\nResultado do Group by com Order by:")
print(result3.head())



Resultado do Group by com Order by:
   Cliente_cpf          cpf  Qdte_Total  Valor_Total
0  12057103021  12057103021        50.0        100.0
2  14719263038  14719263038        30.0         80.0
1  12057193021  12057193021         0.0          0.0
3  18315064088  18315064088         0.0          0.0


In [19]:
# Filtrar nome de animal que contenha a letra "o"
result4 = pd.merge(animal, cliente, left_on='Cliente_cpf', right_on='cpf', how='left') \
            .rename(columns={'nome': 'Nome_Animal', 'nome_cliente': 'Nome_Cliente'}) \
            [animal['nome'].str.contains('o', case=False, na=False)]

print("\nResultado do filtro LIKE:")
print(result4.head())



Resultado do filtro LIKE:
    id   nome_x   especie  idade  peso  Cliente_cpf          cpf    nome_y  \
1    2     Dory     Peixe      3   0.1  12057193021  12057193021    Forond   
2    3  Bolinha  Cachorro      2  10.4  12057103021  12057103021    Bumbwe   
3    4   Pipoca  Papagaio      1   0.9  12057103021  12057103021    Bumbwe   
8    9    Apolo  Cachorro      7  30.0  66739827052  66739827052   Irloaim   
10  11      Zoe    Coelho      8   4.4  79601875093  79601875093  Nusriogu   

               fone               email  
1   (21) 99456-7821    forond@email.com  
2   (11) 99876-5432    bumbwe@email.com  
3   (11) 99876-5432    bumbwe@email.com  
8   (51) 99765-4321   irloaim@email.com  
10  (61) 99654-3210  nusriogu@email.com  


In [20]:
# Subconsulta com operador IN: Selecionar vendas realizadas por clientes específicos
clientes_selecionados = ['12345678900', '98765432100']

result5 = pd.merge(venda, cliente, left_on='Cliente_cpf', right_on='cpf', how='inner') \
            [cliente['cpf'].isin(clientes_selecionados)]

print("\nResultado da subconsulta com operador IN:")
print(result5.head())



Resultado da subconsulta com operador IN:
Empty DataFrame
Columns: [id, Funcionario_cpf, Cliente_cpf, pagamento_id, Data_hora, cpf, nome, fone, email]
Index: []


C:\Users\WTINFO PC\AppData\Local\Temp\ipykernel_11412\2157594113.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result5 = pd.merge(venda, cliente, left_on='Cliente_cpf', right_on='cpf', how='inner') \


In [21]:
# Subconsulta com operador EXISTS: Selecionar funcionários que realizaram vendas
funcionarios_com_vendas = funcionario[funcionario['cpf'].isin(venda['Funcionario_cpf'])]

print("\nResultado da subconsulta com operador EXISTS:")
print(funcionarios_com_vendas.head())



Resultado da subconsulta com operador EXISTS:
          cpf            nome         funcao
2   123456789      João Silva    Veterinário
6   456789123  Pedro Oliveira       Auxiliar
10  789123456     Ana Pereira  Administrador
12  987654321    Maria Santos      Atendente


In [22]:
# Consulta externa: Mostrar todas as vendas, mesmo aquelas que não possuem itens
result7 = pd.merge(venda, itens_venda, left_on='id', right_on='Venda_idVenda', how='left') \
            [['id', 'Funcionario_cpf', 'Cliente_cpf', 'pagamento_id', 'Data_hora', 'Produto_id']]

print("\nResultado da consulta externa (LEFT JOIN):")
print(result7.head())



Resultado da consulta externa (LEFT JOIN):
   id  Funcionario_cpf  Cliente_cpf  pagamento_id           Data_hora  \
0   1        123456789  12057103021             1 2023-05-19 10:30:00   
1   2        987654321  12057193021             2 2023-05-19 11:00:00   
2   3        456789123  14719263038             3 2023-05-19 11:30:00   
3   4        789123456  18315064088             4 2023-05-19 12:00:00   

   Produto_id  
0         1.0  
1         NaN  
2         2.0  
3         NaN  


In [23]:
# Consulta: Contar o número total de vendas realizadas por cada cliente
result8 = pd.merge(venda, cliente, left_on='Cliente_cpf', right_on='cpf', how='left') \
            .groupby(['Cliente_cpf', 'nome']) \
            .agg(Total_Vendas=('id', 'count')) \
            .reset_index()

print("\nResultado da consulta com agregações (count):")
print(result8.head())



Resultado da consulta com agregações (count):
   Cliente_cpf    nome  Total_Vendas
0  12057103021  Bumbwe             1
1  12057193021  Forond             1
2  14719263038  Reaion             1
3  18315064088   Geurt             1


In [22]:
# Fechando a conexão
if connection:
    connection.close()
